Chroma
similarity_search
- Returns List of documents most similar to the query text.
- k (int) – Number of results to return. Defaults to 4.

In [29]:
# HERE ARE THE VERSION NUMBERS THAT WORKED FOR ME:
# CAREFUL WITH PYDANTIC, DO IT LAST SINCE CHROMA AND LANGCHAIN AUTO INSTALL IT AS A DEPENDENCY
# Use this to install specific versions numbers:
# !pip install package_name==0.3.26
import chromadb
print(chromadb.__version__)
import langchain
print(langchain.__version__)
import pydantic
print(pydantic.__version__)

0.5.3
0.2.6
2.8.2


In [30]:
import chromadb

In [31]:
# from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

from langchain_google_genai import GoogleGenerativeAIEmbeddings

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # type: ignore

### Load Document and Split (even if under context window, still recommended)

In [32]:
# load the document and split it into chunks
path = '../langchain-course-main/June 2024 Update/01-Data-Connections/'
loader = TextLoader(path+"some_data/FDR_State_of_Union_1944.txt")
documents = loader.load()

In [33]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

In [34]:
print(type(docs))
print(len(docs))

<class 'list'>
15


In [35]:
print(docs[0].page_content)

This Nation in the past two years has become an active partner in the world's greatest war against human slavery.

We have joined with like-minded people in order to defend ourselves in a world that has been gravely threatened with gangster rule.

But I do not think that any of us Americans can be content with mere survival. Sacrifices that we and our allies are making impose upon us all a sacred obligation to see to it that out of this war we and our children will gain something better than mere survival.

We are united in determination that this war shall not be followed by another interim which leads to new disaster- that we shall not repeat the tragic errors of ostrich isolationism—that we shall not repeat the excesses of the wild twenties when this Nation went for a joy ride on a roller coaster which ended in a tragic crash.

When Mr. Hull went to Moscow in October, and when I went to Cairo and Teheran in November, we knew that we were in agreement with our allies in our common de

In [36]:
print(docs[1].page_content)

And right here I want to address a word or two to some suspicious souls who are fearful that Mr. Hull or I have made "commitments" for the future which might pledge this Nation to secret treaties, or to enacting the role of Santa Claus.

To such suspicious souls—using a polite terminology—I wish to say that Mr. Churchill, and Marshal Stalin, and Generalissimo Chiang Kai-shek are all thoroughly conversant with the provisions of our Constitution. And so is Mr. Hull. And so am I.

Of course we made some commitments. We most certainly committed ourselves to very large and very specific military plans which require the use of all Allied forces to bring about the defeat of our enemies at the earliest possible time.

But there were no secret treaties or political or financial commitments.

The one supreme objective for the future, which we discussed for each Nation individually, and for all the United Nations, can be summed up in one word: Security.

And that means not only physical security 

In [37]:
print(docs[2].page_content)

All our allies have learned by bitter experience that real development will not be possible if they are to be diverted from their purpose by repeated wars—or even threats of war.

China and Russia are truly united with Britain and America in recognition of this essential fact:

The best interests of each Nation, large and small, demand that all freedom-loving Nations shall join together in a just and durable system of peace. In the present world situation, evidenced by the actions of Germany, Italy, and Japan, unquestioned military control over disturbers of the peace is as necessary among Nations as it is among citizens in a community. And an equally basic essential to peace is a decent standard of living for all individual men and women and children in all Nations. Freedom from fear is eternally linked with freedom from want.

There are people who burrow through our Nation like unseeing moles, and attempt to spread the suspicion that if other Nations are encouraged to raise their sta

In [38]:
print(docs[3].page_content)

Returning from my journeyings, I must confess to a sense of "let-down" when I found many evidences of faulty perspective here in Washington. The faulty perspective consists in overemphasizing lesser problems and thereby underemphasizing the first and greatest problem.

The overwhelming majority of our people have met the demands of this war with magnificent courage and understanding. They have accepted inconveniences; they have accepted hardships; they have accepted tragic sacrifices. And they are ready and eager to make whatever further contributions are needed to win the war as quickly as possible- if only they are given the chance to know what is required of them.

However, while the majority goes on about its great work without complaint, a noisy minority maintains an uproar of demands for special favors for special groups. There are pests who swarm through the lobbies of the Congress and the cocktail bars of Washington, representing these special groups as opposed to the basic int

### Connect to OpenAI for Embeddings

### Pass Embeddings and Docs into Chroma

In [39]:
# load it into Chroma
db = Chroma.from_documents(docs, gemini_embeddings,persist_directory='./speech_embedding_db')

### Save the new embeddings to disk

In [8]:
# Helpful to force a save
# db.persist()

### Load Embeddings from Disk

In [40]:
db_connection = Chroma(persist_directory='./speech_embedding_db/',embedding_function=gemini_embeddings)

In [41]:
# WATCH THE VIDEO TO TRULY UNDERSTAND WHY YOU MAY NOT WANT TO DO DIRECT QUESTIONS!
new_doc = "What did FDR say about the cost of food law?"

In [42]:
sim_docs = db_connection.similarity_search(new_doc)

In [43]:
print(len(sim_docs))
print(type(sim_docs[0]))

4
<class 'langchain_core.documents.base.Document'>


In [44]:
print(sim_docs[0].page_content)

That is the way to fight and win a war—all out—and not with half-an-eye on the battlefronts abroad and the other eye-and-a-half on personal, selfish, or political interests here at home.

Therefore, in order to concentrate all our energies and resources on winning the war, and to maintain a fair and stable economy at home, I recommend that the Congress adopt:

(1) A realistic tax law—which will tax all unreasonable profits, both individual and corporate, and reduce the ultimate cost of the war to our sons and daughters. The tax bill now under consideration by the Congress does not begin to meet this test.

(2) A continuation of the law for the renegotiation of war contracts—which will prevent exorbitant profits and assure fair prices to the Government. For two long years I have pleaded with the Congress to take undue profits out of war.

(3) A cost of food law—which will enable the Government (a) to place a reasonable floor under the prices the farmer may expect for his production; and

In [45]:
print(sim_docs[1].page_content)

(4) Early reenactment of. the stabilization statute of October, 1942. This expires June 30, 1944, and if it is not extended well in advance, the country might just as well expect price chaos by summer.

(5) A national service law- which, for the duration of the war, will prevent strikes, and, with certain appropriate exceptions, will make available for war production or for any other essential services every able-bodied adult in this Nation.

These five measures together form a just and equitable whole. I would not recommend a national service law unless the other laws were passed to keep down the cost of living, to share equitably the burdens of taxation, to hold the stabilization line, and to prevent undue profits.

The Federal Government already has the basic power to draft capital and property of all kinds for war purposes on a basis of just compensation.

As you know, I have for three years hesitated to recommend a national service act. Today, however, I am convinced of its necess

In [46]:
print(sim_docs[2].page_content)

If we analyze American history impartially, we cannot escape the fact that in our past we have not always forgotten individual and selfish and partisan interests in time of war—we have not always been united in purpose and direction. We cannot overlook the serious dissensions and the lack of unity in our war of the Revolution, in our War of 1812, or in our War Between the States, when the survival of the Union itself was at stake.

In the first World War we came closer to national unity than in any previous war. But that war lasted only a year and a half, and increasing signs of disunity began to appear during the final months of the conflict.

In this war, we have been compelled to learn how interdependent upon each other are all groups and sections of the population of America.

Increased food costs, for example, will bring new demands for wage increases from all war workers, which will in turn raise all prices of all things including those things which the farmers themselves have to

In [47]:
print(sim_docs[3].page_content)

I ask the Congress to explore the means for implementing this economic bill of rights- for it is definitely the responsibility of the Congress so to do. Many of these problems are already before committees of the Congress in the form of proposed legislation. I shall from time to time communicate with the Congress with respect to these and further proposals. In the event that no adequate program of progress is evolved, I am certain that the Nation will be conscious of the fact.

Our fighting men abroad- and their families at home- expect such a program and have the right to insist upon it. It is to their demands that this Government should pay heed rather than to the whining demands of selfish pressure groups who seek to feather their nests while young Americans are dying.

The foreign policy that we have been following—the policy that guided us at Moscow, Cairo, and Teheran—is based on the common sense principle which was best expressed by Benjamin Franklin on July 4, 1776: "We must al

In [48]:
embedded_text = gemini_embeddings.embed_query(new_doc)

In [54]:
print(type(embedded_text))
print(len(embedded_text))
print(type(embedded_text[0]))
print(embedded_text[0])

<class 'list'>
768
<class 'float'>
0.02614925242960453


In [55]:

result = db_connection.similarity_search_by_vector(embedding=embedded_text,k=2,)

In [58]:
print(result)
print(type(result))
print(len(result))

[Document(metadata={'source': '../langchain-course-main/June 2024 Update/01-Data-Connections/some_data/FDR_State_of_Union_1944.txt'}, page_content='That is the way to fight and win a war—all out—and not with half-an-eye on the battlefronts abroad and the other eye-and-a-half on personal, selfish, or political interests here at home.\n\nTherefore, in order to concentrate all our energies and resources on winning the war, and to maintain a fair and stable economy at home, I recommend that the Congress adopt:\n\n(1) A realistic tax law—which will tax all unreasonable profits, both individual and corporate, and reduce the ultimate cost of the war to our sons and daughters. The tax bill now under consideration by the Congress does not begin to meet this test.\n\n(2) A continuation of the law for the renegotiation of war contracts—which will prevent exorbitant profits and assure fair prices to the Government. For two long years I have pleaded with the Congress to take undue profits out of wa

In [60]:
print(result[0].page_content)

That is the way to fight and win a war—all out—and not with half-an-eye on the battlefronts abroad and the other eye-and-a-half on personal, selfish, or political interests here at home.

Therefore, in order to concentrate all our energies and resources on winning the war, and to maintain a fair and stable economy at home, I recommend that the Congress adopt:

(1) A realistic tax law—which will tax all unreasonable profits, both individual and corporate, and reduce the ultimate cost of the war to our sons and daughters. The tax bill now under consideration by the Congress does not begin to meet this test.

(2) A continuation of the law for the renegotiation of war contracts—which will prevent exorbitant profits and assure fair prices to the Government. For two long years I have pleaded with the Congress to take undue profits out of war.

(3) A cost of food law—which will enable the Government (a) to place a reasonable floor under the prices the farmer may expect for his production; and

## Add New Document

In [61]:
# load the document and split it into chunks
path = '../langchain-course-main/June 2024 Update/01-Data-Connections/'
loader = TextLoader(path+"some_data/Lincoln_State_of_Union_1862.txt")
documents = loader.load()

In [63]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

Created a chunk of size 608, which is longer than the specified 500
Created a chunk of size 539, which is longer than the specified 500
Created a chunk of size 686, which is longer than the specified 500


In [64]:
# load it into Chroma
db = Chroma.from_documents(docs, gemini_embeddings,persist_directory='./speech_embedding_db')

In [65]:
docs = db.similarity_search('slavery')

In [66]:
docs[0].page_content

'Among the friends of the Union there is great diversity of sentiment and of policy in regard to slavery and the African race amongst us. Some would perpetuate slavery; some would abolish it suddenly and without compensation; some would abolish it gradually and with compensation: some would remove the freed people from us, and some would retain them with us; and there are yet other minor diversities. Because of these diversities we waste much strength in struggles among ourselves. By mutual concession we should harmonize and act together. This would be compromise, but it would be compromise among the friends and not with the enemies of the Union. These articles are intended to embody a plan of such mutual concessions. if the plan shall be adopted, it is assumed that emancipation will follow, at least in several of the States.\n\nAs to the first article, the main points are, first, the emancipation; secondly, the length of time for consummating it (thirty-seven years); and, thirdly, the

### Collection class calls: https://docs.trychroma.com/reference/Collection

In [ ]:
# help(db._collection)